In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
hyperparameters = {
    'num_classes': 9,
    'cnn_channels': [32, 64, 128, 256, 512, 9],
    'lr': 1e-4,
    'batch_size': 64,
    'val_split': 0.2,
    'test_split': 0.2,
    'num_workers': 4,
    'max_epochs': 50,
}

In [ ]:
import pandas as pd
from dataset import SudokuDataModule

data = pd.read_csv('data/sudoku_small.csv')
dataset = SudokuDataModule(data, hyperparameters)

In [ ]:
from model import SudokuSolverCNN
from lightning.pytorch.utilities.model_summary import ModelSummary

model = SudokuSolverCNN(hyperparameters)
ModelSummary(model, max_depth=2)

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.tuner.tuning import Tuner
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor

trainer = Trainer(
    max_epochs=10,
    logger=TensorBoardLogger('logs', name='sudoku-solver', log_graph=True),
    callbacks=[
        ModelCheckpoint(monitor='val_loss', save_top_k=1, mode='min'),
        EarlyStopping(monitor='val_loss', patience=5, mode='min'),
        LearningRateMonitor(logging_interval='step')
    ],
    enable_checkpointing=True,
    enable_progress_bar=True,
)

In [ ]:
trainer.fit(model, datamodule=dataset)

In [ ]:
trainer.test(model, datamodule=dataset)

In [ ]:
import torch


puzzle = "020008004090100600003000010901400008000000302005083007004000920700300850000256403"
puzzle_grid = [int(x) for x in puzzle]
puzzle_tensor = torch.tensor(puzzle_grid, dtype=torch.float32).view(1, 9, 9).unsqueeze(1)
puzzle_tensor

In [ ]:
solution = model(puzzle_tensor)
solution.argmax(dim=1).view(9, 9) + 1